In [0]:
import os
import json
import numpy as np
import pandas as pd
import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from math import sqrt

## 데이터 로드

In [0]:
jdbc_url = "jdbc:sqlserver://1dt-team4-sqlserver.database.windows.net:1433;database=1dt-team4-sqldb"
connection_properties = {
    "user": "azureuser",
    "password": "team4123!@#",
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

df = spark.read.jdbc(url=jdbc_url, table="gold.gold_realscore_pred", properties=connection_properties)
df.display()

## 데이터 전처리

In [0]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# -----------------------
# 전처리 함수
# -----------------------
def drop_mean_columns(df):
    cols_to_drop = [c for c in df.columns if c.endswith('_mean')]
    return df.drop(columns=cols_to_drop, errors='ignore')

def preprocess_input(df_in):
    df = df_in.toPandas().copy()

    # 정확도 열 추가
    if 'correct_cnt' in df.columns and 'items_attempted' in df.columns:
        df['accuracy'] = df['correct_cnt'] / df['items_attempted'].replace(0, np.nan)
        df['accuracy'] = df['accuracy'].fillna(0)
    else:
        df['accuracy'] = 0

    # 결측치 0 처리
    df = df.fillna(0)

    # '_mean' 컬럼 제거
    df = drop_mean_columns(df)

    # 범주형 변수 (있으면 자동 감지)
    categorical_candidates = ['gender', 'grade']
    categorical_cols = [c for c in categorical_candidates if c in df.columns]

    return df, categorical_cols

# -----------------------
# 전처리 실행
# -----------------------
df, categorical_cols = preprocess_input(df)


## train-test-split

In [0]:
TARGET = "realScore_clean"   # 예측할 target
EXCLUDE_COLS = ['learnerID', 'testID', 'correct_cnt', 'items_attempted']  # 학습에서 제외할 컬럼들
RANDOM_STATE = 42

# feature/target 분리
feature_cols = [c for c in df.columns if c not in ([TARGET] + EXCLUDE_COLS)]
X = df[feature_cols]
y = df[TARGET]

# train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE
)

print("Train:", X_train.shape, "Test:", X_test.shape)
print("Categorical columns:", categorical_cols)
print("Feature columns:", feature_cols[:10], "...")

## 모델 전처리

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from math import sqrt
from sklearn.model_selection import GridSearchCV

# -----------------------
# 전처리 파이프라인
# -----------------------
numeric_cols = [c for c in X_train.columns if c not in categorical_cols]

num_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value=0))
])

cat_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer(transformers=[
    ('num', num_pipeline, numeric_cols),
    ('cat', cat_pipeline, categorical_cols)
])

## 모델 학습

In [0]:
# -----------------------
# 모델 후보
# -----------------------
pipelines = {
    'dt': Pipeline(steps=[('preproc', preprocessor),
                          ('model', DecisionTreeRegressor(random_state=RANDOM_STATE))]),
    'rf': Pipeline(steps=[('preproc', preprocessor),
                          ('model', RandomForestRegressor(random_state=RANDOM_STATE, n_jobs=-1))])
}

param_grids = {
    'dt': {'model__max_depth': [3, 5, 7]},
    'rf': {'model__n_estimators': [50, 100],
           'model__max_depth': [5, 10, None]}
}

# -----------------------
# 학습 & 평가
# -----------------------
results = {}
for name, pipe in pipelines.items():
    print(f"\nTraining {name} ...")
    gs = GridSearchCV(pipe, param_grids[name], cv=3,
                      scoring='neg_mean_squared_error', n_jobs=-1)
    gs.fit(X_train, y_train)
    
    best = gs.best_estimator_
    preds = best.predict(X_test)
    
    rmse = sqrt(mean_squared_error(y_test, preds))
    mae = mean_absolute_error(y_test, preds)
    r2 = r2_score(y_test, preds)
    
    results[name] = {
        'best_model': best,
        'best_params': gs.best_params_,
        'rmse': rmse,
        'mae': mae,
        'r2': r2
    }
    print(f"{name} best params: {gs.best_params_}")
    print(f"→ RMSE={rmse:.4f}, MAE={mae:.4f}, R2={r2:.4f}")

## 모델 선택

In [0]:
# -----------------------
# 베스트 모델 선택
# -----------------------
best_name = min(results.keys(), key=lambda k: results[k]['rmse'])
best_model = results[best_name]['best_model']
print("\n선택된 모델:", best_name, results[best_name])


## 추가

In [0]:
# ===== 테스트셋 예측 추출 =====
TARGET = "realScore_clean"  # 위에서 사용한 것과 동일해야 함

# 1) 예측 수행
y_pred = best_model.predict(X_test)

# 2) realScore_true / realScore_pred 로 컬럼명 지정
pred_df = pd.DataFrame({
    "realScore_true": y_test.values,
    "realScore_pred": y_pred
}, index=X_test.index)

# 3) 절대 오차
pred_df["error"] = (pred_df["realScore_true"] - pred_df["realScore_pred"]).abs()

# 4) 식별자(있으면) 붙이기: 원본 df에서 테스트 인덱스 행의 메타 컬럼만 조인
id_cols = [c for c in ["learnerID", "testID", "gender", "grade"] if c in df.columns]
if id_cols:
    pred_df = df.loc[X_test.index, id_cols].join(pred_df)

# 5) 확인: 상위 10건
print(f"rows={len(pred_df)}")
display(pred_df.head(10))


In [0]:
# testID 오름차순 정렬(보조 키로 learnerID까지 정렬하고 싶으면 주석 해제)
if "testID" not in pred_df.columns:
    raise KeyError("pred_df에 'testID' 컬럼이 없습니다. 위에서 id_cols 조인 단계가 실행됐는지 확인하세요.")

# (간단) 문자열/숫자 무엇이든 기본 오름차순
df= pred_df.sort_values(by=["testID"], ascending=True)
# pred_df_sorted = pred_df.sort_values(by=["testID", "learnerID"], ascending=[True, True])  # 보조 키 사용 시

display(df.head(20))  # 원하는 만큼 표시


In [0]:
# === grade_percentile 계산 (예측값 realScore_pred 기준) ===
import numpy as np
import pandas as pd

# 0) 필수/입력 컬럼 확인 및 realScore_pred 확보
for col in ["testID", "grade"]:
    if col not in df.columns:
        raise KeyError(f"df에 '{col}' 컬럼이 없습니다. 전처리/조인 단계를 확인하세요.")

if "realScore_pred" not in df.columns:
    # X_test 기반 예측이 pred_df에 있다면 조인
    if "pred_df" in globals() and "realScore_pred" in pred_df.columns:
        df = df.join(pred_df[["realScore_pred"]], how="left")
    else:
        raise KeyError("df에 'realScore_pred'가 없습니다. pred_df 조인 또는 예측 생성 후 다시 실행하세요.")

# 1) 각 testID가 단일 학년 시험인지 판별 (SQL의 n_test_grade = n_test 조건과 동치)
test_single_grade = (df.groupby("testID")["grade"].nunique() == 1)
is_single = df["testID"].map(test_single_grade)

# 2) PERCENT_RANK() 구현: (rank - 1) / (N - 1), NaN은 제외
def percent_rank_ignore_na(s: pd.Series) -> pd.Series:
    out = pd.Series(np.nan, index=s.index)
    mask = s.notna()
    n = mask.sum()
    if n <= 1:
        out[mask] = 0.0
        return out
    ranks = s[mask].rank(method="min", ascending=True)
    out[mask] = (ranks - 1) / (n - 1)
    return out

# 3) 두 가지 분모로 백분위 계산 (예측값 사용)
#    - 단일 학년 시험(testID) → grade 단위
#    - 복수 학년 시험        → (testID, grade) 단위
pr_by_grade_pred = df.groupby("grade")["realScore_pred"].transform(percent_rank_ignore_na)
pr_by_test_grade_pred = df.groupby(["testID", "grade"])["realScore_pred"].transform(percent_rank_ignore_na)

# 4) 선택 적용 + 0~100 스케일
df["grade_percentile_calc"] = np.where(is_single, pr_by_grade_pred, pr_by_test_grade_pred) * 100.0

# 5) 소수 둘째 자리 '버림' (T-SQL: ROUND(x, 2, 1) 동일 동작)
tmp = df["grade_percentile_calc"].clip(lower=0, upper=100)
df["grade_percentile"] = np.floor(tmp * 100) / 100

# 6) testID 오름차순 정렬 후 표시 (learnerID 있으면 함께 표시)
print("✅ df에 'grade_percentile' 컬럼(예측값 기반) 추가 완료")
cols_to_show = ["learnerID", "testID", "grade", "realScore_pred", "grade_percentile"]
existing = [c for c in cols_to_show if c in df.columns]

df_sorted = df.sort_values(by="testID", ascending=True).reset_index(drop=True)
display(df_sorted[existing].head(12))


In [0]:
# === achievement_grade (NTILE(5) by testID, ORDER BY grade_percentile DESC, theta_clean DESC, learnerID ASC) ===
import numpy as np
import pandas as pd

# 0) 기준 컬럼 선택: grade_percentile
base_col = None
for c in ["grade_percentile"]:
    if c in df.columns:
        base_col = c
        break
if base_col is None:
    raise KeyError("df에 'grade_percentile' 컬럼이 없습니다. 먼저 계산하세요.")

# 1) 보조 정렬 컬럼(동점 처리) 준비: theta_clean / learnerID 없으면 빈 값으로 채움
if "theta_clean" not in df.columns:
    df["theta_clean"] = np.nan
if "learnerID" not in df.columns:
    df["learnerID"] = ""

# 2) 그룹별 NTILE(5) 부여 (SQL NTILE과 동일하게 행 순서 기반 분배)
def assign_ntile(group: pd.DataFrame) -> pd.DataFrame:
    # 점수 높은 순 → theta 높은 순 → learnerID 사전순
    g = group.sort_values([base_col, "theta_clean", "learnerID"],
                          ascending=[False, False, True], kind="mergesort").copy()
    n = len(g)
    if n == 0:
        return g
    pos = np.arange(1, n + 1)                   # 1..n
    ntile = np.ceil(pos * 5 / n).astype(int)    # NTILE(5)
    ntile = np.clip(ntile, 1, 5)
    g["achievement_grade"] = pd.Series(ntile, index=g.index).map({
        1: "A", 2: "B", 3: "C", 4: "D", 5: "F"
    })
    return g

df = (df.groupby("testID", group_keys=False)
        .apply(assign_ntile)
        .reset_index(drop=True))

# 3) 보기 좋게 정렬 및 미리보기
df = df.sort_values(by="testID", ascending=True).reset_index(drop=True)
display(df[["testID", "learnerID", "grade", base_col, "achievement_grade"]].head(20))


## MLFlow 실험 기록 & 모델 저장

In [0]:
import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient

# -----------------------
# MLflow experiment 설정
# -----------------------
EXPERIMENT_NAME = "/Users/1dt003@msacademy.msai.kr/real_score_experiment"
MODEL_NAME = "real-score-model"
mlflow.set_experiment(EXPERIMENT_NAME)

# -----------------------
# MLflow 실행 (run) 시작
# -----------------------
with mlflow.start_run(run_name=f"real-score-v1-{best_name}") as run:
    run_id = run.info.run_id
    
    # 어떤 모델이 선택되었는지 기록
    mlflow.log_param("selected_model", best_name)
    
    # 선택된 모델의 하이퍼파라미터 기록
    for p, v in results[best_name]['best_params'].items():
        mlflow.log_param(p, v)
    
    # 모든 모델의 평가 지표 기록
    for name, res in results.items():
        mlflow.log_metric(f"{name}_rmse", res['rmse'])
        mlflow.log_metric(f"{name}_mae", res['mae'])
        mlflow.log_metric(f"{name}_r2", res['r2'])
    
    # 선택된 모델의 최종 지표 기록
    mlflow.log_metric("rmse", results[best_name]['rmse'])
    mlflow.log_metric("mae", results[best_name]['mae'])
    mlflow.log_metric("r2", results[best_name]['r2'])
    
    # === 수정된 부분 ===
    # 1. MLflow에 자동으로 모델 로그
    mlflow.sklearn.log_model(
        sk_model=best_model,
        artifact_path="model",
        registered_model_name=MODEL_NAME
    )
    print("모델 MLflow에 자동 등록 완료")